# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. FirstName LastName

### Grupo GG
<dl>
    <dt>HH horas (33.3%)</dt>
    <dd>istxxxxxxxx FirstName LastName</dd>
    <dt>HH horas (33.3%)</dt>
    <dd>istxxxxxxxx FirstName LastName</dd>
    <dt>HH horas (33.3%)</dt>
    <dd>istxxxxxxxx FirstName LastName</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%%sql

DROP TABLE IF EXISTS customer CASCADE;
DROP TABLE IF EXISTS orders CASCADE;
DROP TABLE IF EXISTS pay CASCADE;
DROP TABLE IF EXISTS employee CASCADE;
DROP TABLE IF EXISTS process CASCADE;
DROP TABLE IF EXISTS department CASCADE;
DROP TABLE IF EXISTS workplace CASCADE;
DROP TABLE IF EXISTS works CASCADE;
DROP TABLE IF EXISTS office CASCADE;
DROP TABLE IF EXISTS warehouse CASCADE;
DROP TABLE IF EXISTS product CASCADE;
DROP TABLE IF EXISTS contains CASCADE;
DROP TABLE IF EXISTS supplier CASCADE;
DROP TABLE IF EXISTS delivery CASCADE;

CREATE TABLE customer(
cust_no INTEGER PRIMARY KEY,
name VARCHAR(80) NOT NULL,
email VARCHAR(254) UNIQUE NOT NULL,
phone VARCHAR(15),
address VARCHAR(255)
);

CREATE TABLE orders(
order_no INTEGER PRIMARY KEY,
cust_no INTEGER NOT NULL REFERENCES customer,
date DATE NOT NULL
--order_no must exist in contains
);

CREATE TABLE pay(
order_no INTEGER PRIMARY KEY REFERENCES orders,
cust_no INTEGER NOT NULL REFERENCES customer
);

CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
--age must be >=18
);

CREATE TABLE process(
ssn VARCHAR(20) REFERENCES employee,
order_no INTEGER REFERENCES orders,
PRIMARY KEY (ssn, order_no)
);

CREATE TABLE department(
name VARCHAR PRIMARY KEY
);

CREATE TABLE workplace(
address VARCHAR PRIMARY KEY,
lat NUMERIC(8, 6) NOT NULL,
long NUMERIC(9, 6) NOT NULL,
UNIQUE(lat, long)
--address must be in warehouse or office but not both
);

CREATE TABLE office(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE warehouse(
address VARCHAR(255) PRIMARY KEY REFERENCES workplace
);

CREATE TABLE works(
ssn VARCHAR(20) REFERENCES employee,
name VARCHAR(200) REFERENCES department,
address VARCHAR(255) REFERENCES workplace,
PRIMARY KEY (ssn, name, address)
);

CREATE TABLE product(
SKU VARCHAR(25) PRIMARY KEY,
name VARCHAR(200) NOT NULL,
description VARCHAR,
price NUMERIC(10, 2) NOT NULL,
ean NUMERIC(13) UNIQUE
);

CREATE TABLE contains(
order_no INTEGER REFERENCES orders,
SKU VARCHAR(25) REFERENCES product,
qty INTEGER,
PRIMARY KEY (order_no, SKU)
);

CREATE TABLE supplier(
TIN VARCHAR(20) PRIMARY KEY,
name VARCHAR(200),
address VARCHAR(255),
SKU VARCHAR(25) REFERENCES product,
date DATE
);

CREATE TABLE delivery(
address VARCHAR(255) REFERENCES warehouse,
TIN VARCHAR(20) REFERENCES supplier,
PRIMARY KEY (address, TIN)
);

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%%sql
-- populate.sql
INSERT INTO customer VALUES
  (1, 'Customer 1', 'customer1@gmail.com', '912234234', 'Rua Carlos Cruz 10 1701-123 Lisboa'),
  (2, 'Customer 2', 'customer2@gmail.com', '922222222', 'Rua Inês Marques 21 1878-123 Lisboa')
  (3, 'Customer 3', 'customer3@gmail.com', '913232321', 'Rua António Cruz 33 1776-123 Lisboa'),
  (4, 'Customer 4', 'customer4@gmail.com', '919090909', 'Rua João dos Anjos 44 1240-123 Lisboa'),
  (5, 'Customer 5', 'customer5@gmail.com', '923131231', 'Rua dos abacaxis 20 1120-123 Lisboa'),
  (6, 'Customer 6', 'customer6@gmail.com', '912313446', 'Rua das pereiras 43 1230-123 Lisboa');
  

    INSERT INTO order VALUES
  (1, 1, '2023-05-10'),
  (2, 2,'2023-05-15'),
  (3, 2, '2023-05-20'),
  (4, 3, '2023-05-25'),
  (5, 4, '2023-05-30'),
  (6, 5, '2023-06-01'),
  (7, 6, '2022-05-25'),
  (8, 4, '2022-05-30'),
  (9, 5, '2022-06-01'),
  

  INSERT INTO pay VALUES
 (1, 1),
 (2, 2),
 (3, 2),
 (4, 3);

 INSERT INTO employee VALUES
  (12, 1234, '1999-06-12', 'Employee 1'),
  (21, 9876, '1999-06-11', 'Employee 2'),
  (32, 2234, '1999-06-17', 'Employee 3'),
  (44, 1824, '2000-01-01', 'Employee 4'),
  (51, 3234, '2000-03-01', 'Employee 5'),
  (66, 2876, '2000-01-06', 'Employee 6'),

  INSERT INTO process VALUES
  (12, 1),
  (21, 2),
  (32, 3),
  (44, 4),
  (51, 5),
  (66, 6),
  (66, 7),
  (66, 8),
  (66, 9);

  INSERT INTO department VALUES
  ('HR'),
  ('Marketing'),
  ('Research'),
  ('IT');

INSERT INTO workplace VALUES
  ('Rua Caio 25 1700-123 Lisboa', 20.1000, -74.2121),
  ('Rua Torga 10 1750-120 Lisboa', 10.2323, -55.1000),
  ('Rua Caio 10 1700-124 Lisboa', 21.1000, -72.2121),
  ('Rua Torga 25 1750-121 Lisboa', 12.2323, -52.1000)
  ('Rua Caio 5 1700-123 Lisboa', 22.1000, -73.2121),
  ('Rua Torga 11 1750-123 Lisboa', 15.2323, -56.1000);

INSERT INTO office VALUES
  ('Rua Caio 25 1700-123 Lisboa'),
  ('Rua Torga 11 1750-123 Lisboa'),
  ('Rua Caio 10 1700-124 Lisboa');

INSERT INTO warehouse VALUES
  ('Rua Torga 10 1700-124 Lisboa'),
  ('Rua Torga 25 1750-121 Lisboa'),
  ('Rua Caio 5 1700-123 Lisboa');

INSERT INTO works VALUES
  (12, 'IT', 'Rua Caio 25 1700-123 Lisboa'),
  (21, 'HR', 'Rua Torga 11 1750-123 Lisboa'),
  (32, 'Marketing', 'Rua Caio 10 1700-124 Lisboa'),
  (44, 'HR', 'Rua Caio 5 1700-123 Lisboa'),
  (51, 'IT', 'Rua Torga 25 1750-121 Lisboa'),
  (66, 'Research', 'Rua Torga 10 1700-124 Lisboa')

INSERT INTO product VALUES
  (1, 'Product A', 'Description of Product A', 9.99, '1234567890911'),
  (2, 'Product B', 'Description of Product B', 10.50, '9876543210285'),
  (3, 'Product C', 'Description of Product C', 9.99, '1234567891016'),
  (4, 'Product D', 'Description of Product D', 10.50, '9876543211749'),
  (5, 'Product A', 'Description of Product E', 9.99, '1234567892098'),
  (6, 'Product B', 'Description of Product F', 10.50, '9876543212523'),
  (7, 'Product C', 'Description of Product G', 9.99, '1234567893211'),
  (8, 'Product D', 'Description of Product H', 10.50, '9876543213213');

INSERT INTO contains VALUES
  (1, 2, 2),
  (2, 3, 4),
  (3, 4, 1),
  (3, 1, 25),
  (4, 5, 3),
  (5, 6, 2),
  (6, 7, 5),
  (6, 8, 10),
  (7, 1, 5),
  (8, 3, 10),
  (9, 5, 10);

  INSERT INTO supplier VALUES
  (123456789, 'Supplier X', 'Rua dos Marcos 5 1900-123 Lisboa', 1, '2023-01-10'),
  (987654321, 'Supplier Y', 'Rua Carlos Meireles 10 1900-023 Lisboa', 2, '2023-02-15'),
  (123456781, 'Supplier Z', 'Rua Manuel Silva 10 1900-120 Lisboa', 3, '2023-01-10'),
  (987654322, 'Supplier W', 'Rua dos Marcos 10 1900-124 Lisboa', 4, '2023-02-15')
  (123456782, 'Supplier I', 'Rua Manuel Silva 11 1900-123 Lisboa', 5, '2023-01-10'),
  (987654323, 'Supplier P', 'Rua Pedro Pais 10 1900-523 Lisboa', 6, '2023-02-15'),
  (123456783, 'Supplier L', 'Rua dos Marcos 12 1900-126 Lisboa', 7, '2023-01-10'),
  (987654324, 'Supplier M', 'Rua Pedro Pais 20 1900-140 Lisboa', 8, '2023-02-15');
 
INSERT INTO delivery VALUES
  ('Rua Caio 5 1700-123 Lisboa', 123456789),
  ('Rua Torga 10 1700-124 Lisboa', 987654321),
  ('Rua Torga 10 1700-124 Lisboa', 123456781),
  ('Rua Torga 10 1700-124 Lisboa', 987654323),
  ('Rua Torga 25 1750-121 Lisboa', 987654324),
  ('Rua Torga 25 1750-121 Lisboa', 123456783);
  

## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
-- (RI-1)
CREATE TABLE employee(
ssn VARCHAR(20) PRIMARY KEY,
TIN VARCHAR(20) UNIQUE NOT NULL,
bdate DATE,
name VARCHAR NOT NULL
CONSTRAINT CHECK ((EXTRACT(YEAR FROM bdate))<2005)
);

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql
-- (RI-2)

CREATE OR REPLACE FUNCTION cancel_office_creation_func() RETURNS TRIGGER AS
$$
BEGIN
    IF NEW.address IN (SELECT address FROM warehouse) THEN
        RAISE EXCEPTION 'Ja existe warehouse com o endereco: % ',NEW.address;
    END IF;
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE TRIGGER cancel_office_creation BEFORE INSERT OR UPDATE ON office
FOR EACH ROW EXECUTE FUNCTION cancel_office_creation_func();


CREATE OR REPLACE FUNCTION cancel_warehouse_creation_func() RETURNS TRIGGER AS
$$
BEGIN
    IF NEW.address IN (SELECT address FROM office) THEN
        RAISE EXCEPTION 'Ja existe office com o endereco: % ',NEW.address;
    END IF;
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE TRIGGER cancel_warehouse_creation BEFORE INSERT OR UPDATE ON warehouse
FOR EACH ROW EXECUTE FUNCTION cancel_warehouse_creation_func();


(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql
-- (RI-3)

CREATE OR REPLACE FUNCTION order_creation_func() RETURNS TRIGGER AS
$$
BEGIN
    IF NEW.order_no NOT IN (SELECT order_no FROM contains) THEN
        INSERT INTO contains (NEW.order_no,null,null);
    END IF;
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE TRIGGER order_creation_trigger BEFORE INSERT OR UPDATE ON orders
FOR EACH ROW EXECUTE FUNCTION order_creation_func();


CREATE FUNCTION contains_count (o_no VARCHAR)
RETURNS INTEGER
AS
$$
DECLARE o_count INTEGER;
BEGIN
SELECT COUNT(*) INTO o_count
FROM contains
WHERE order_no = o_no;
RETURN o_count;
END
$$
LANGUAGE plpgsql;

CREATE OR REPLACE FUNCTION order_delete_func() RETURNS TRIGGER AS
$$
BEGIN
    IF (contains_count(OLD.order_no)==1) THEN
        RAISE EXCEPTION 'Order com order_no: % tem que existir em contains ',OLD.order_no;
    END IF;
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE TRIGGER order_delete_trigger BEFORE DELETE ON contains
FOR EACH ROW EXECUTE FUNCTION order_delete_func();


## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql
-- SELECT ...
SELECT name,cust_no
FROM costumer 
    NATURAL JOIN pay
    NATURAL JOIN contains
    NATURAL JOIN (SELECT sku,price FROM product)
GROUP BY ssn
HAVING SUM(qty*price)>= ALL
    (SELECT SUM(qty*price)
    FROM costumer 
    NATURAL JOIN pay
    NATURAL JOIN contains
    NATURAL JOIN (SELECT sku,price FROM product)
    GROUP BY ssn)


2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql
-- SELECT ...
WITH orderDays AS(
    SELECT COUNT(DISTINCT(date)) AS workingDays FROM orders WHERE EXTRACT(YEAR FROM date)=2022),
employeeWorkingDays AS(
    SELECT name,TIN,COUNT(DISTINCT(date)) AS workingDays
    FROM employee 
    NATURAL JOIN process
    NATURAL JOIN orders (SELECT * FROM orders WHERE EXTRACT(YEAR FROM date)=2022)
    GROUP BY TIN
)
SELECT name FROM employeeWorkingDays WHERE workingDays IN (SELECT workingDAYS FROM orderDays);


3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql
-- SELECT ...
SELECT EXTRACT(MONTH FROM date) AS month, COUNT (DISTINCT(order_no)) AS numUnpaidOrders
FROM orders WHERE order_no NOT IN(SELECT order_no FROM pay) AND EXTRACT(YEAR FROM date)=2022
GROUP BY EXTRACT(MONTH FROM date) AS month;
  


## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql
-- CREATE VIEW ...
CREATE VIEW product_sales AS
SELECT sku, order_no, qty, (qty * price) AS total_price, year, month, day_of_month, day_of_week, city
FROM contains NATURAL JOIN pay
              NATURAL JOIN (SELECT sku, price FROM  product)
              NATURAL JOIN (SELECT order_no, EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month, EXTRACT(DAY FROM date) AS day_of_month, EXTRACT(DAYOFWEEK FROM date) AS day_of_week FROM orders) 
              NATURAL JOIN (SELECT cust_no, RIGHT(address, CHARINDEX (' ', REVERSE(address))-1) AS city FROM customer);

## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

...

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql
-- SELECT ...
SELECT sku, city, month, day_of_month, day_of_week, SUM(qty) AS total_qty, SUM(total_price) AS total_value
FROM product_sales
WHERE year = 2022
GROUP BY ROLLUP(sku, city, month, day_of_month, day_of_week)


2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql
-- SELECT ...
SELECT month, day_of_week, AVG(total_price) AS average_daily_sales
FROM product_sales
WHERE year = 2022
GROUP BY ROLLUP(month, day_of_week)

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

...

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

...